# Purpose

The purpose of this notebook is to compute the list of processes in a Linux scenario and the proportion of their occurrence in the trace.

Note that this notebook is distributed for distribution purpose, and not intended for production use.

# Perimeter

- Linux 64-bit scenarios for which 
  [OSSI is supported by REVEN](http://doc.tetrane.com/professional/latest/Project-Manager/Workflow/Get-OSSI-Linux.html#vm-requirements).
- Supported REVEN version: 2.8.2+


# Limitations

- The `current_task_offset` variable must be read from the `kernel_description.json` file of the target scenario.
  This file can be found in the replay directory of the target scenario.
- Possible changes of the value in `gs_base` and `kernel_gs_base` are ignored, with the assumption that the value
  never changes and is simply swapped between `gs_base` and `kernel_gs_base`. If this assumption is incorrect, then
  process changes will be missed.

## Parameters

In [ ]:
# Server connection
host = 'localhost'
port = 41607

# Required field offset from gs_base for current_task, find it in kernel_description.json
current_task_offset = 0xd300  # Correct value for the Fedora VM that is distributed by Tetrane

In [1]:
import reven2
server = reven2.RevenServer(host, port)
print(server.trace.transition_count)

4257686967


In [61]:
# As in Linux 64bit, gs_base and kernel_gs_base don't seem to change but are just swap 
# we can use just the memory history to track the changes of process
def get_process_changes():
    tr = server.trace.transition(0)
    ctx_before = tr.context_before()
    current_process = ctx_before.ossi.process()
        
    yield tr

    gs_base = ctx_before.read(reven2.arch.msr.gs_base)

    if gs_base == 0:
        gs_base = ctx_before.read(reven2.arch.msr.kernel_gs_base)

    # FIXME: If MMU change the physical address of this linear address it won't work
    for memaccess in server.trace.memory_accesses(
        reven2.address.LinearAddress(gs_base + current_task_offset),
        8,
        from_transition=tr,
        operation=reven2.memhist.MemoryAccessOperation.Write
    ):
        memaccess_transition = memaccess.transition
        if current_process.pid != memaccess_transition.context_after().ossi.process().pid:
            yield memaccess_transition
        
            tr = memaccess_transition
            current_process = tr.context_after().ossi.process()

In [70]:
%%time
%%prun -s time

processes = {}
last_transition_id = 0

try:
    for tr in get_process_changes():
        current_process = tr.context_before().ossi.process()
        next_process = tr.context_after().ossi.process()
        
        current_process_tuple = (current_process.name, current_process.pid)

        if current_process_tuple not in processes:
            processes[current_process_tuple] = 0

        print("Changing to process %s at %s" % (next_process, tr))

        processes[current_process_tuple] += tr.id - last_transition_id
        last_transition_id = tr.id
        
    last_transition_id = server.trace.transition_count
except KeyboardInterrupt:
    print("Interrupted by user")

Changing to process exploit_userspe (1541) at #0 ret 
Changing to process xfce4-terminal (974) at #29672 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process ksoftirqd/0 (7) at #46860 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process kworker/0:1 (1432) at #55342 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process ksoftirqd/0 (7) at #65937 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process xfce4-terminal (974) at #68477 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process exploit_userspe (1541) at #104729 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process xfce4-terminal (974) at #127753 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process exploit_userspe (1541) at #243895 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process xfce4-terminal (974) at #300472 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process exploit_userspe (1541) at #374985 mov qword ptr gs:[rip+0x7efe7d71], rbx
Changing to process xfce4-terminal (974) at #39

In [71]:
print("Processes:")
for (process_name, process_pid), nb_instrs in reversed(sorted(processes.items(), key=lambda x: x[1])):
    print(" - %s (pid: %d): %d instructions (%.4f%% of the trace)" % (
        process_name, process_pid, 
        nb_instrs, 
        (nb_instrs / last_transition_id) * 100
    ))

Processes:
 - sudo (pid: 1541): 4041627346 instructions (94.9254% of the trace)
 - tee (pid: 1543): 83993966 instructions (1.9728% of the trace)
 - exploit_userspe (pid: 1541): 51378101 instructions (1.2067% of the trace)
 - sudo (pid: 1543): 37237307 instructions (0.8746% of the trace)
 - systemd-journal (pid: 373): 18116746 instructions (0.4255% of the trace)
 - in:imjournal (pid: 493): 3833283 instructions (0.0900% of the trace)
 - unix_chkpwd (pid: 1542): 3728068 instructions (0.0876% of the trace)
 - Xorg (pid: 614): 2360334 instructions (0.0554% of the trace)
 - dbus-daemon (pid: 491): 2134845 instructions (0.0501% of the trace)
 - sssd_nss (pid: 545): 1681198 instructions (0.0395% of the trace)
 - rcu_sched (pid: 8): 1640368 instructions (0.0385% of the trace)
 - xfce4-terminal (pid: 974): 1424720 instructions (0.0335% of the trace)
 - kworker/0:2 (pid: 88): 875519 instructions (0.0206% of the trace)
 - kworker/u2:2 (pid: 86): 825146 instructions (0.0194% of the trace)
 - sudo (